In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import os

# Set the base directory (change this to your project root if needed)
base_dir = r"c:\Users\subbu\OneDrive\Desktop\FINALYEAR\media"

# Ensure proper model path and file loading
model_path = os.path.join(base_dir, "posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite")

# Check if the model exists
if not os.path.exists(model_path):
    print(f"PoseNet model could not be loaded: Could not open '{model_path}'")
else:
    print(f"PoseNet model found at {model_path}")
    # Load the PoseNet model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

# Ensure correct paths for person and clothing images
person_image_path = os.path.join(base_dir, "testproj.jpg")
dress_image_path = os.path.join(base_dir,  "dresss.jpg")

# Check if person image exists
if not os.path.exists(person_image_path):
    print(f"Failed to load person image: {person_image_path}")
    person_image = None
else:
    person_image = cv2.imread(person_image_path)
    if person_image is not None:
        print("Person image loaded successfully.")
    else:
        print(f"Failed to read the person image from: {person_image_path}")

# Check if clothing image exists
if not os.path.exists(dress_image_path):
    print(f"Failed to load clothing image: {dress_image_path}")
    clothing_image = None
else:
    clothing_image = cv2.imread(dress_image_path)
    if clothing_image is not None:
        print("Clothing image loaded successfully.")
    else:
        print(f"Failed to read the clothing image from: {dress_image_path}")

# Proceed only if both images and the model are loaded
if person_image is not None and clothing_image is not None:

    # Example: Dummy keypoints for testing (replace with actual PoseNet inference output)
    keypoints = {
        0: {
            5: [100, 150],  # Left shoulder
            6: [200, 150],  # Right shoulder
            11: [100, 300], # Left hip
            12: [200, 300]  # Right hip
        }
    }

    # Extract relevant keypoints
    try:
        left_shoulder = keypoints[0][5]
        right_shoulder = keypoints[0][6]
        left_hip = keypoints[0][11]
        right_hip = keypoints[0][12]

        # Define source points in the clothing image (you can adjust these points)
        source_points = np.float32([[0, 0], [clothing_image.shape[1], 0], [0, clothing_image.shape[0]]])
        
        # Define destination points on the person's body (using keypoints)
        destination_points = np.float32([left_shoulder, right_shoulder, left_hip])

        # Compute the transformation matrix for warping the clothing image onto the person image
        M = cv2.getAffineTransform(source_points, destination_points)

        # Apply affine transformation to warp the clothing image
        warped_clothing = cv2.warpAffine(clothing_image, M, (person_image.shape[1], person_image.shape[0]))

        # Create a mask for the warped clothing (you can refine this for blending)
        clothing_mask = warped_clothing[:, :, 0] > 0  # Non-black pixels in clothing

        # Overlay the clothing on the person image
        result_image = person_image.copy()
        result_image[clothing_mask] = warped_clothing[clothing_mask]

        # Show the final image
        cv2.imshow('Virtual Try-On Result', result_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    except IndexError as e:
        print(f"Failed to get keypoints: {e}")

else:
    print("Ensure both images are properly loaded before proceeding.")#loads

PoseNet model found at c:\Users\subbu\OneDrive\Desktop\FINALYEAR\media\posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite
Person image loaded successfully.
Clothing image loaded successfully.


In [2]:
import cv2
import numpy as np
import tensorflow as tf
from scipy.interpolate import Rbf

# Load the PoseNet model
model_path = r"c:\Users\subbu\OneDrive\Desktop\FINALYEAR\media\posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Function to detect keypoints
def detect_keypoints(interpreter, image):
    # Prepare the input image for PoseNet
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_shape = input_details[0]['shape']

    # Resize and normalize the input image
    image_resized = cv2.resize(image, (input_shape[2], input_shape[1]))
    input_data = np.expand_dims(image_resized.astype(np.float32), axis=0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    
    # Extract keypoints
    keypoints = interpreter.get_tensor(output_details[0]['index'])[0]
    return keypoints

# Load images
person_image_path = r"c:\Users\subbu\OneDrive\Desktop\FINALYEAR\media\testproj.jpg"
dress_image_path = r"c:\Users\subbu\OneDrive\Desktop\FINALYEAR\media\dresss.jpg"
person_image = cv2.imread(person_image_path)
clothing_image = cv2.imread(dress_image_path)

# Detect keypoints in the person image
keypoints = detect_keypoints(interpreter, person_image)

# Use detected keypoints for left/right shoulders and hips
left_shoulder = keypoints[0][5]
right_shoulder = keypoints[0][6]
left_hip = keypoints[0][11]
right_hip = keypoints[0][12]

# Resize clothing to match the shoulder width
shoulder_width = int(np.linalg.norm(np.array(left_shoulder) - np.array(right_shoulder)))
aspect_ratio = clothing_image.shape[1] / clothing_image.shape[0]
new_height = int(shoulder_width / aspect_ratio)
clothing_image = cv2.resize(clothing_image, (shoulder_width, new_height))

# Define clothing source and target points for TPS
source_points = np.float32([
    [0, 0],
    [clothing_image.shape[1], 0],
    [0, clothing_image.shape[0]],
    [clothing_image.shape[1], clothing_image.shape[0]]
])
destination_points = np.float32([left_shoulder, right_shoulder, left_hip, right_hip])

# Thin Plate Spline transformation using RBF
def apply_tps_transform(source, destination, image_shape):
    x = np.array([p[0] for p in destination], dtype=np.float32)
    y = np.array([p[1] for p in destination], dtype=np.float32)
    z_x = np.array([p[0] for p in source], dtype=np.float32)
    z_y = np.array([p[1] for p in source], dtype=np.float32)

    # RBF Interpolation
    rbf_x = Rbf(x, y, z_x, function='thin_plate')
    rbf_y = Rbf(x, y, z_y, function='thin_plate')

    grid_x, grid_y = np.meshgrid(np.arange(image_shape[1]), np.arange(image_shape[0]))
    map_x = rbf_x(grid_x, grid_y).astype(np.float32)
    map_y = rbf_y(grid_x, grid_y).astype(np.float32)

    return map_x, map_y

# Generate transformation mapping using TPS
map_x, map_y = apply_tps_transform(source_points, destination_points, person_image.shape)

# Apply warp using remap for a flexible transformation
warped_clothing = cv2.remap(clothing_image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

# Create a binary mask from the warped clothing
_, clothing_mask = cv2.threshold(cv2.cvtColor(warped_clothing, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY)

# Refine edges by blurring
clothing_mask = cv2.GaussianBlur(clothing_mask, (5, 5), 0)

# Invert mask for person background
inverse_mask = cv2.bitwise_not(clothing_mask)

# Mask the person image background
person_bg = cv2.bitwise_and(person_image, person_image, mask=inverse_mask)

# Isolate the clothing region from the warped clothing image
clothing_fg = cv2.bitwise_and(warped_clothing, warped_clothing, mask=clothing_mask)

# Combine images
result_image = cv2.add(person_bg, clothing_fg)

# Display the final result
cv2.imshow("Virtual Try-On Result", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


IndexError: index 11 is out of bounds for axis 0 with size 9